Домашняя работа по теме "Базовые понятия статистики"

In [ ]:
import pandas as pd
import numpy as np

Задание 1. Загрузка данных

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [ ]:
horse_colic = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv',
                          header=None,
                          na_values='?')[[0, 1, 3, 4, 5, 6, 10, 22]]

In [ ]:
horse_colic.rename(columns={0:'surgery',
                            1:'age',
                            3:'rectal_temperature',
                            4:'pulse',
                            5:'respiratory_rate',
                            6:'temperature_of_extremities',
                            10:'pain',
                            22:'outcome'},
                   inplace=True)
horse_colic.head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [ ]:
#расчет моды для качественных данных
horse_colic.mode()[['surgery', 'age', 'temperature_of_extremities', 'pain', 'outcome']]

,surgery,age,temperature_of_extremities,pain,outcome
0,1.0,1,3.0,3.0,1.0


In [ ]:
#расчет базовых статистик для количественных данных
horse_colic.describe()[['rectal_temperature',	'pulse',	'respiratory_rate']]

,rectal_temperature,pulse,respiratory_rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


In [ ]:
#проверка  данных на соответствие описанию показала, что в столбце Age есть некорректные значения: вместо 2 указано 9
horse_colic[horse_colic['age'] > 2].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
9,2.0,9,38.3,90.0,NaN,1.0,5.0,1.0
13,2.0,9,38.0,92.0,28.0,1.0,1.0,2.0
16,1.0,9,NaN,128.0,36.0,3.0,4.0,2.0
23,1.0,9,38.3,130.0,60.0,NaN,2.0,1.0


In [ ]:
#исправляю выбросы по столбцу Age
horse_colic.loc[(horse_colic['age'] > 2), 'age'] = 2
horse_colic[horse_colic['age'] == 2].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
9,2.0,2,38.3,90.0,NaN,1.0,5.0,1.0
13,2.0,2,38.0,92.0,28.0,1.0,1.0,2.0
16,1.0,2,NaN,128.0,36.0,3.0,4.0,2.0
23,1.0,2,38.3,130.0,60.0,NaN,2.0,1.0


In [ ]:
# при аналогичной проверке других столбцов с качественными данными некорректных значений не выявлено
horse_colic[horse_colic['outcome'] <= 0]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome


In [ ]:
# поиск выбросов через межквартильный размах (для количественных данных)
def get_outliers(df, column):
  """Возвращает датафрейм с выбросами по определенному столбцу базируясь на межквартильном размахе"""
  q1 = df[column].quantile(0.25)
  q3 = df[column].quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - (1.5 * iqr)
  upper_bound = q3 + (1.5 * iqr)
  outliers = (df[~df[column]
                 .between(lower_bound, upper_bound, inclusive='both')]
              .dropna(subset=[column])
              )
  return outliers

In [ ]:
outliers_for_rect_temp = get_outliers(horse_colic, 'rectal_temperature')
outliers_for_rect_temp.head()
#получаем 14 строк с выбросами

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
54,2.0,1,40.3,114.0,36.0,3.0,2.0,3.0
75,1.0,2,39.7,100.0,NaN,3.0,2.0,3.0
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0


In [ ]:
outliers_for_pulse = get_outliers(horse_colic, 'pulse')
outliers_for_pulse.head()
#получаем 5 строк с выбросами

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,2,38.6,160.0,20.0,3.0,3.0,2.0
255,1.0,2,38.8,184.0,84.0,1.0,4.0,2.0
275,1.0,2,38.8,150.0,50.0,1.0,5.0,2.0


In [ ]:
outliers_for_resp_rate = get_outliers(horse_colic, 'respiratory_rate')
outliers_for_resp_rate.head()
#получаем 17 строк с выбросами

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
39,1.0,2,39.2,146.0,96.0,NaN,NaN,2.0
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
82,1.0,2,38.1,100.0,80.0,3.0,3.0,1.0
84,1.0,1,37.8,60.0,80.0,1.0,2.0,1.0


Вывод по выбросам: т.к. датафреймы с выбросами небольшие, можно увидеть, что только 3 строки встречаются больше чем в одном датафрейме (с индексами 3, 41, 255). Их можно исключить из основного датафрейма без ущерба для данных.

Исключение всех выбросов, выявленных на основе межквартильного размаха приведет к потере 11% данных. К тому же по своим значениям они похожи на естественные выбросы. Можно дополнительно поизучать их отдельно.

In [ ]:
horse_colic.drop(index=[3, 41, 255], inplace=True)
horse_colic.head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0


Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [ ]:
horse_colic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     296 non-null    float64
 1   age                         297 non-null    int64  
 2   rectal_temperature          237 non-null    float64
 3   pulse                       273 non-null    float64
 4   respiratory_rate            239 non-null    float64
 5   temperature_of_extremities  242 non-null    float64
 6   pain                        243 non-null    float64
 7   outcome                     296 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 20.9 KB


In [ ]:
(horse_colic.isna().mean() * 100).round(2)

surgery                        0.34
age                            0.00
rectal_temperature            20.20
pulse                          8.08
respiratory_rate              19.53
temperature_of_extremities    18.52
pain                          18.18
outcome                        0.34
dtype: float64

Значительные пропуски

In [ ]:
horse_colic[(horse_colic['rectal_temperature'].isna() == True) &
            (horse_colic['pulse'].isna() == True) &
            (horse_colic['respiratory_rate'].isna() == True)
            ]
# т.к. все расчеты проводятся для количественных данных,
# строки с пропущенными значениями в этих столбцах (одновременно) можно исключить без ущерба для расчетов
# их заполнение может привести к искажениям, а имеющихся в других столбцах данных недостаточно для заполнения

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
28,1.0,1,NaN,NaN,NaN,NaN,NaN,2.0
52,2.0,1,NaN,NaN,NaN,1.0,1.0,1.0
56,1.0,1,NaN,NaN,NaN,NaN,NaN,1.0
74,1.0,2,NaN,NaN,NaN,NaN,NaN,2.0
78,1.0,1,NaN,NaN,NaN,3.0,5.0,2.0
93,2.0,1,NaN,NaN,NaN,3.0,5.0,2.0
136,1.0,1,NaN,NaN,NaN,3.0,5.0,2.0
160,1.0,1,NaN,NaN,NaN,NaN,4.0,1.0
174,2.0,1,NaN,NaN,NaN,3.0,4.0,2.0


In [ ]:
horse_colic.dropna(how='all', subset=['rectal_temperature', 'pulse', 'respiratory_rate'], inplace=True)
horse_colic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     281 non-null    float64
 1   age                         282 non-null    int64  
 2   rectal_temperature          237 non-null    float64
 3   pulse                       273 non-null    float64
 4   respiratory_rate            239 non-null    float64
 5   temperature_of_extremities  236 non-null    float64
 6   pain                        236 non-null    float64
 7   outcome                     281 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 19.8 KB


Пропуски в surgery и outcome

In [ ]:
# заполнение пропусков начинаю с качественных данных
horse_colic[horse_colic['surgery'].isna() == True]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
132,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


In [ ]:
horse_colic[horse_colic['outcome'].isna() == True]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
132,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


In [ ]:
# т.к. это качественные данные и только один пропуск, можно заполнить его модой
horse_colic['surgery'] = horse_colic['surgery'].fillna(horse_colic['surgery'].mode()[0])
horse_colic['outcome'] = horse_colic['outcome'].fillna(horse_colic['outcome'].mode()[0])

Пропуски в temperature_of_extremities

In [ ]:
horse_colic[horse_colic['temperature_of_extremities'].isna() == True].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
17,2.0,1,37.5,48.0,24.0,NaN,NaN,1.0
23,1.0,2,38.3,130.0,60.0,NaN,2.0,1.0
25,2.0,1,37.8,60.0,42.0,NaN,NaN,1.0


In [ ]:
# в temperature_of_extremities значительное количество пропусков
# значения столбца коррелируют с повышенной ректальной температурой
# заполнение модой полностью может привести к искажениям при заполнении пропусков в rectal_temperature
# поэтому модой заполняю только в части данных с неповышенной температурой
def get_temperature_of_extremities (row):
  """Возвращает значение температуры конечностей в зависимости от ректальной температуры"""
  if row['rectal_temperature'] > 38.3:
    return 2
  else:
    return 3

In [ ]:
horse_colic['temperature_of_extremities'] = horse_colic['temperature_of_extremities'].fillna(horse_colic.apply(get_temperature_of_extremities, axis=1))

Пропуски в pain

In [ ]:
horse_colic[horse_colic['pain'].isna() == True].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
4,2.0,1,37.3,104.0,35.0,3.0,NaN,2.0
7,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
17,2.0,1,37.5,48.0,24.0,3.0,NaN,1.0
19,2.0,1,39.4,110.0,35.0,4.0,NaN,1.0
24,1.0,1,38.1,60.0,12.0,3.0,NaN,1.0


In [ ]:
# в pain значительное количество пропусков
# заполнение модой полностью может привести к искажениям при заполнении пропусков в pulse и respiratory_rate
# поэтому модой заполняю только в части данных с отклонениями от нормы пульса
def get_pain (row):
  """Возвращает значение уровня боли в зависимости от нормальности других показателей"""
  if row['pulse'] <= 50 and row['pulse'] >= 30:
    return 1
  else:
    return 3

In [ ]:
horse_colic['pain'] = horse_colic['pain'].fillna(horse_colic.apply(get_pain, axis=1))

Пропуски в pulse

In [ ]:
horse_colic[horse_colic['pulse'].isna() == True]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
58,1.0,1,NaN,NaN,20.0,4.0,5.0,2.0
83,1.0,1,38.0,NaN,24.0,3.0,5.0,2.0
115,2.0,1,NaN,NaN,40.0,3.0,3.0,1.0
117,2.0,1,39.5,NaN,NaN,3.0,3.0,3.0
126,1.0,1,38.7,NaN,NaN,3.0,5.0,1.0
151,1.0,1,39.3,NaN,NaN,4.0,4.0,2.0
159,2.0,1,NaN,NaN,12.0,1.0,2.0,1.0
182,1.0,1,38.0,NaN,24.0,3.0,5.0,2.0
204,1.0,1,39.2,NaN,23.0,3.0,4.0,1.0


In [ ]:
# среднее и медианнное значения pulse сильно отличаются в разных возрастных группах
# поэтому при заполнении дополнительно следует группировать по возрасту
horse_colic.groupby('age').describe()[['pulse']]

pulse                                                        
     count        mean        std   min   25%    50%    75%    max
age                                                               
1    254.0   67.456693  23.933953  30.0  48.0   60.0   84.0  140.0
2     19.0  116.631579  23.935586  84.0  96.0  120.0  133.0  160.0

In [ ]:
# т.к. болезненные поражения могут влять на скорость сердцебиения,
# можно сгруппировать по pain и заполнить медианным значением pulse по группе (для устойчивости к выбросам)
horse_colic['pulse'] = horse_colic['pulse'].fillna(horse_colic
                                                   .groupby(['age', 'pain'])['pulse']
                                                   .transform('median')
                                                   )

Пропуски в rectal_temperature

In [ ]:
horse_colic[horse_colic['rectal_temperature'].isna() == True].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
7,1.0,1,NaN,60.0,NaN,3.0,3.0,2.0
8,2.0,1,NaN,80.0,36.0,3.0,4.0,3.0
16,1.0,2,NaN,128.0,36.0,3.0,4.0,2.0
34,1.0,1,NaN,100.0,30.0,3.0,5.0,1.0
35,2.0,1,NaN,104.0,24.0,4.0,4.0,3.0


In [ ]:
# т.к. ректальная температура коррелирует с температурой конечностей,
# группирую по temperature_of_extremities и заполняю медианным значением rectal_temperature по группе
# дополнительно группирую по surgery, т.к. температура может также зависеть от шока
horse_colic['rectal_temperature'] = horse_colic['rectal_temperature'].fillna(horse_colic
                                                                             .groupby(['surgery', 'temperature_of_extremities'])['rectal_temperature']
                                                                             .transform('median')
                                                                             )

Пропуски в respiratory_rate

In [ ]:
horse_colic[horse_colic['respiratory_rate'].isna() == True].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
7,1.0,1,38.1,60.0,NaN,3.0,3.0,2.0
9,2.0,2,38.3,90.0,NaN,1.0,5.0,1.0
29,2.0,1,37.7,48.0,NaN,2.0,1.0,1.0
32,1.0,1,37.2,60.0,NaN,2.0,3.0,1.0
37,1.0,1,37.8,72.0,NaN,3.0,5.0,1.0


In [ ]:
# среднее и медианнное значения respiratory_rate сильно отличаются в разных возрастных группах
# поэтому при заполнении дополнительно следует группировать по возрасту
horse_colic.groupby('age').describe()[['respiratory_rate']]

respiratory_rate                                                    
               count       mean        std   min   25%   50%   75%   max
age                                                                     
1              222.0  28.238739  15.251192   8.0  18.0  24.0  36.0  96.0
2               17.0  50.117647  23.026839  20.0  30.0  44.0  68.0  96.0

In [ ]:
# по информации из интернета учащенное дыхание у лошадей может быть связано со стрессом и болезнью,
# поэтому группирую по pain и заполняю медианным значением respiratory_rate по группе
horse_colic['respiratory_rate'] = horse_colic['respiratory_rate'].fillna(horse_colic
                                                                         .groupby(['age', 'pain'])['respiratory_rate']
                                                                         .transform('median')
                                                                         )

In [ ]:
# до заполнения пропусков
horse_colic.describe()[['rectal_temperature',	'pulse',	'respiratory_rate']]

,rectal_temperature,pulse,respiratory_rate
count,237.000000,273.000000,239.000000
mean,38.157806,70.879121,29.794979
std,0.731200,26.979500,16.834208
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.000000
50%,38.200000,64.000000,24.000000
75%,38.500000,88.000000,36.000000
max,40.800000,160.000000,96.000000


In [ ]:
# после заполнения пропусков
horse_colic.describe()[['rectal_temperature',	'pulse',	'respiratory_rate']]

,rectal_temperature,pulse,respiratory_rate
count,282.000000,282.000000,282.000000
mean,38.158333,71.095745,29.301418
std,0.673396,26.643282,15.769456
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,20.000000
50%,38.100000,64.000000,24.000000
75%,38.500000,88.000000,35.750000
max,40.800000,160.000000,96.000000


In [ ]:
horse_colic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     282 non-null    float64
 1   age                         282 non-null    int64  
 2   rectal_temperature          282 non-null    float64
 3   pulse                       282 non-null    float64
 4   respiratory_rate            282 non-null    float64
 5   temperature_of_extremities  282 non-null    float64
 6   pain                        282 non-null    float64
 7   outcome                     282 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 19.8 KB


Домашняя работа выполнена.